In [1]:
import pandas as pd
import os
import json
from datetime import datetime
import subprocess

def procesar_excel_y_actualizar_html():
    base_path = r"C:\Users\Jorge Vasquez\Ranking"
    ruta_excel = os.path.join(base_path, "CS_AVANCE DE ASESORES.xlsx")
    ruta_html = os.path.join(base_path, "index.html")
    
    print(f"Trabajando desde: {base_path}")
    print(f"Excel: {ruta_excel}")
    print(f"HTML: {ruta_html}")
    
    try:
        # Verificar que el archivo Excel existe
        if not os.path.exists(ruta_excel):
            print(f"ERROR: No se encuentra el archivo Excel en {ruta_excel}")
            return
        
        # Obtener todas las hojas disponibles del Excel
        excel_file = pd.ExcelFile(ruta_excel)
        hojas_disponibles = excel_file.sheet_names
        print(f"Hojas disponibles: {hojas_disponibles}")
        
        # Filtrar hojas que siguen el patrón MES_AÑO
        meses_validos = []
        for hoja in hojas_disponibles:
            if '_' in hoja and any(mes in hoja.upper() for mes in ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 
                                                                  'JULIO', 'AGOSTO', 'SETIEMBRE', 'SEPTIEMBRE', 'OCTUBRE', 
                                                                  'NOVIEMBRE', 'DICIEMBRE']):
                meses_validos.append(hoja)
        
        print(f"Meses válidos encontrados: {meses_validos}")
        
        # Extraer años y meses únicos
        años_unicos = sorted(set(hoja.split('_')[1] for hoja in meses_validos), reverse=True)
        meses_unicos = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 
                       'JULIO', 'AGOSTO', 'SETIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
        
        print(f"Años disponibles: {años_unicos}")
        print(f"Meses disponibles: {meses_unicos}")
        
        # Procesar datos para cada mes y obtener lista de todos los asesores
        todos_los_datos = {}
        todos_los_asesores = set()
        
        for hoja in meses_validos:
            print(f"\nProcesando hoja: {hoja}")
            df = pd.read_excel(ruta_excel, sheet_name=hoja, header=None)
            asesores_data = extraer_datos_completos(df, hoja)
            todos_los_datos[hoja] = asesores_data
            
            # Agregar asesores a la lista global
            for asesor in asesores_data:
                todos_los_asesores.add(asesor['nombre'])
            
            print(f"  - {len(asesores_data)} asesores procesados")
        
        print(f"\nTotal de asesores únicos encontrados: {len(todos_los_asesores)}")
        
        # Generar el HTML con todas las funcionalidades
        generar_html_completo(todos_los_datos, ruta_html, meses_unicos, años_unicos, meses_validos, sorted(todos_los_asesores))
        
        # Cambiar al directorio del proyecto para operaciones Git
        os.chdir(base_path)
        
        # Realizar commit y push en Git
        realizar_git_operations()
        
        print("Proceso completado exitosamente!")
        print(f"Archivo HTML generado en: {ruta_html}")
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

def extraer_datos_completos(df, nombre_hoja):
    """Extrae datos completos incluyendo RECUPERO, META y datos diarios"""
    asesores_data = []
    
    # La fila 7 contiene los encabezados
    fila_encabezados = 7
    
    # Identificar las columnas correctas basándonos en los encabezados
    row_7 = df.iloc[fila_encabezados]
    
    # Buscar índices de columnas importantes
    col_orden = None
    col_al_ac = None
    col_clasificacion = None
    col_aser = None
    col_supervisor = None
    col_recupero = None
    col_meta = None
    columnas_fechas = {}  # Diccionario para almacenar columnas de fechas
    
    for i, cell in enumerate(row_7):
        if pd.notna(cell):
            cell_str = str(cell).strip().upper()
            if 'ORDEN' in cell_str:
                col_orden = i
            elif 'AL_AC' in cell_str:
                col_al_ac = i
            elif 'CLASIFICACIÓN' in cell_str or 'CLASIFICACION' in cell_str:
                col_clasificacion = i
            elif 'ASER_' in cell_str:
                col_aser = i
            elif 'SUPERVISOR' in cell_str:
                col_supervisor = i
            elif 'RECUPERO' in cell_str:
                col_recupero = i
            elif 'META' in cell_str:
                col_meta = i
            elif '-' in cell_str and any(mes in cell_str for mes in ['ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC']):
                # Es una columna de fecha
                columnas_fechas[i] = cell_str
    
    # Si no encontramos las columnas por nombre, usar las posiciones estándar
    if col_orden is None: col_orden = 7
    if col_al_ac is None: col_al_ac = 8
    if col_clasificacion is None: col_clasificacion = 9
    if col_aser is None: col_aser = 10
    if col_supervisor is None: col_supervisor = 16
    if col_recupero is None: col_recupero = 17
    if col_meta is None: col_meta = 18
    
    # Procesar TODAS las filas desde la 8 hasta el final del DataFrame
    start_data_row = 8
    total_filas = len(df)
    
    filas_procesadas = 0
    filas_con_datos = 0
    filas_sin_nombre = 0
    max_filas_sin_datos = 10
    
    for idx in range(start_data_row, total_filas):
        row = df.iloc[idx]
        filas_procesadas += 1
        
        # Verificar si tenemos datos en la columna ASER_
        if (col_aser < len(row) and pd.notna(row[col_aser]) and 
            str(row[col_aser]).strip() not in ['', 'NaN', 'None', 'nan']):
            
            nombre = str(row[col_aser]).strip()
            filas_sin_nombre = 0  # Resetear contador de filas sin nombre
            
            # Obtener porcentaje de AL_AC
            porcentaje = None
            if col_al_ac < len(row) and pd.notna(row[col_al_ac]):
                try:
                    valor = row[col_al_ac]
                    if isinstance(valor, (int, float)):
                        porcentaje = float(valor) * 100
                    else:
                        valor_str = str(valor).strip()
                        if es_porcentaje_valido(valor_str):
                            porcentaje = float(valor_str.replace('%', '').replace(',', '.')) * 100
                except Exception as e:
                    pass
            
            # Obtener nombre del supervisor
            supervisor = ""
            if col_supervisor < len(row) and pd.notna(row[col_supervisor]):
                supervisor = str(row[col_supervisor]).strip()
            
            # Obtener RECUPERO
            recupero = None
            if col_recupero < len(row) and pd.notna(row[col_recupero]):
                try:
                    recupero = float(row[col_recupero])
                except:
                    pass
            
            # Obtener META
            meta = None
            if col_meta < len(row) and pd.notna(row[col_meta]):
                try:
                    meta = float(row[col_meta])
                except:
                    pass
            
            # Obtener datos diarios
            datos_diarios = {}
            for col_idx, fecha_str in columnas_fechas.items():
                if col_idx < len(row) and pd.notna(row[col_idx]):
                    try:
                        valor = float(row[col_idx])
                        datos_diarios[fecha_str] = valor
                    except:
                        datos_diarios[fecha_str] = 0
                else:
                    datos_diarios[fecha_str] = 0
            
            # Si tenemos nombre y porcentaje, agregar
            if nombre and porcentaje is not None:
                clasificacion = determinar_clasificacion(porcentaje)
                asesores_data.append({
                    'nombre': nombre,
                    'porcentaje': round(porcentaje, 1),
                    'clasificacion': clasificacion,
                    'supervisor': supervisor,
                    'recupero': recupero,
                    'meta': meta,
                    'datos_diarios': datos_diarios,
                    'orden': row[col_orden] if col_orden < len(row) and pd.notna(row[col_orden]) else None
                })
                filas_con_datos += 1
        else:
            # Celda sin nombre
            filas_sin_nombre += 1
            if filas_sin_nombre >= max_filas_sin_datos and filas_con_datos > 0:
                break
    
    return asesores_data

def determinar_clasificacion(porcentaje):
    """Determina la clasificación según el porcentaje"""
    if porcentaje > 100:
        return ">100%"
    elif porcentaje > 70:
        return ">70%"
    elif porcentaje > 40:
        return ">40%"
    else:
        return ">0%"

def es_porcentaje_valido(cadena):
    """Verifica si una cadena representa un porcentaje válido"""
    cadena_limpia = str(cadena).replace('%', '').replace(',', '.').strip()
    try:
        valor = float(cadena_limpia)
        return 0 <= valor <= 200
    except:
        return False

def generar_html_completo(todos_los_datos, ruta_html, meses_unicos, años_unicos, meses_validos, lista_asesores):
    """Genera el HTML con todas las funcionalidades incluyendo Periodo de Prueba"""
    
    # Determinar mes y año actual más reciente
    mes_actual, año_actual = obtener_mes_año_actual(meses_validos)
    
    # Obtener datos del mes actual
    clave_actual = f"{mes_actual}_{año_actual}"
    asesores_data = todos_los_datos.get(clave_actual, [])
    asesores_data.sort(key=lambda x: x['porcentaje'], reverse=True)
    
    # Contar asesores por categoría
    contadores = {
        ">100%": len([a for a in asesores_data if a['clasificacion'] == ">100%"]),
        ">70%": len([a for a in asesores_data if a['clasificacion'] == ">70%"]),
        ">40%": len([a for a in asesores_data if a['clasificacion'] == ">40%"]),
        ">0%": len([a for a in asesores_data if a['clasificacion'] == ">0%"])
    }
    
    # Crear opciones de los selectores
    opciones_mes = ""
    for mes in meses_unicos:
        selected = "selected" if mes == mes_actual else ""
        opciones_mes += f'<option value="{mes}" {selected}>{mes.title()}</option>\n'
    
    opciones_año = ""
    for año in años_unicos:
        selected = "selected" if año == año_actual else ""
        opciones_año += f'<option value="{año}" {selected}>{año}</option>\n'
    
    # Crear opciones para el autocompletado de asesores
    opciones_asesores = "".join([f'<option value="{asesor}">' for asesor in lista_asesores])
    
    html_content = f'''<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Evolutivo de Alcance por Asesor</title>
    <style>
        :root {{
            --verde-oscuro: #1b5e20;
            --verde-claro: #c8e6c9;
            --amarillo-oscuro: #FCCF10;
            --amarillo-claro: #FEF6D2;
            --naranja-oscuro: #e65100;
            --naranja-claro: #ffe0b2;
            --rojo-oscuro: #b71c1c;
            --rojo-claro: #ffcdd2;
            --azul-oscuro: #1565c0;
            --azul-claro: #e3f2fd;
            --gris: #f5f5f5;
        }}
        
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }}
        
        body {{
            background-color: #f0f2f5;
            color: #333;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
        }}
        
        header {{
            text-align: center;
            margin-bottom: 30px;
            padding: 20px;
            background: linear-gradient(135deg, #2c3e50, #4a6491);
            color: white;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }}
        
        h1 {{
            font-size: 2.5rem;
            margin-bottom: 10px;
        }}
        
        .selectores-periodo {{
            margin: 20px 0;
            text-align: center;
        }}
        
        .selectores-periodo select {{
            padding: 10px 15px;
            font-size: 1.1rem;
            border: 2px solid #4a6491;
            border-radius: 8px;
            background-color: white;
            cursor: pointer;
            margin: 0 10px;
        }}
        
        .fecha-actualizacion {{
            font-size: 1rem;
            margin-top: 10px;
            opacity: 0.9;
        }}
        
        .busqueda-asesores {{
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            margin: 30px 0;
        }}
        
        .busqueda-asesores h2 {{
            color: #2c3e50;
            margin-bottom: 15px;
            text-align: center;
        }}
        
        .controles-busqueda {{
            display: flex;
            gap: 15px;
            align-items: center;
            flex-wrap: wrap;
            justify-content: center;
        }}
        
        .input-busqueda {{
            flex: 1;
            min-width: 300px;
            padding: 12px 15px;
            font-size: 1rem;
            border: 2px solid #ddd;
            border-radius: 8px;
            background-color: white;
        }}
        
        .boton-busqueda {{
            padding: 12px 25px;
            font-size: 1rem;
            background: var(--azul-oscuro);
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            transition: background 0.3s;
        }}
        
        .boton-busqueda:hover {{
            background: #0d47a1;
        }}
        
        .asesores-seleccionados {{
            margin-top: 20px;
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
        }}
        
        .tag-asesor {{
            background: var(--azul-claro);
            color: var(--azul-oscuro);
            padding: 8px 15px;
            border-radius: 20px;
            display: flex;
            align-items: center;
            gap: 8px;
            font-weight: 500;
        }}
        
        .eliminar-asesor {{
            background: none;
            border: none;
            color: var(--azul-oscuro);
            cursor: pointer;
            font-weight: bold;
            padding: 0;
            width: 20px;
            height: 20px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
        }}
        
        .eliminar-asesor:hover {{
            background: var(--azul-oscuro);
            color: white;
        }}
        
        .periodo-prueba {{
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            margin: 20px 0;
        }}
        
        .periodo-prueba h3 {{
            color: #2c3e50;
            margin-bottom: 15px;
            text-align: center;
        }}
        
        .controles-periodo {{
            display: flex;
            gap: 15px;
            align-items: center;
            flex-wrap: wrap;
            justify-content: center;
            margin-bottom: 20px;
        }}
        
        .input-periodo {{
            padding: 10px 15px;
            font-size: 1rem;
            border: 2px solid #ddd;
            border-radius: 8px;
            background-color: white;
            width: 120px;
            text-align: center;
        }}
        
        .resultados-periodo {{
            margin-top: 20px;
            display: none;
        }}
        
        .tabla-periodo {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
            font-size: 0.9rem;
        }}
        
        .tabla-periodo th {{
            background: var(--azul-oscuro);
            color: white;
            padding: 12px 8px;
            text-align: center;
        }}
        
        .tabla-periodo td {{
            padding: 10px 8px;
            border-bottom: 1px solid #eee;
            text-align: center;
        }}
        
        .resultados-comparacion {{
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            margin-top: 20px;
            display: none;
        }}
        
        .tabla-comparacion {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
            font-size: 0.9rem;
        }}
        
        .tabla-comparacion th {{
            background: var(--azul-oscuro);
            color: white;
            padding: 8px 6px;
            text-align: center;
            white-space: nowrap;
            position: sticky;
            left: 0;
            z-index: 10;
        }}
        
        .tabla-comparacion th.mes-header {{
            background: #2c3e50;
            position: sticky;
            top: 0;
            z-index: 20;
            min-width: 80px;
        }}
        
        .tabla-comparacion td {{
            padding: 6px 4px;
            border-bottom: 1px solid #eee;
            text-align: center;
            min-width: 80px;
        }}
        
        .tabla-comparacion td.asesor-name {{
            background: #f8f9fa;
            font-weight: 600;
            text-align: left;
            position: sticky;
            left: 0;
            z-index: 5;
            min-width: 150px;
            font-size: 0.8rem;
            padding: 6px 8px;
        }}
        
        .tabla-comparacion tr:hover td:not(.asesor-name) {{
            background: #f8f9fa;
        }}
        
        .porcentaje-tabla {{
            font-weight: bold;
            padding: 6px 8px;
            border-radius: 12px;
            color: white;
            text-align: center;
            display: inline-block;
            min-width: 50px;
            font-size: 0.85rem;
        }}
        
        .sin-participacion {{
            color: #999;
            font-style: italic;
        }}
        
        .controles-comparacion {{
            display: flex;
            gap: 10px;
            margin-bottom: 15px;
            flex-wrap: wrap;
        }}
        
        .btn-comparacion {{
            padding: 8px 16px;
            background: var(--azul-oscuro);
            color: white;
            border: none;
            border-radius: 6px;
            cursor: pointer;
            font-size: 0.9rem;
        }}
        
        .btn-comparacion:hover {{
            opacity: 0.9;
        }}
        
        .btn-comparacion.activo {{
            background: var(--verde-oscuro);
        }}
        
        .estado-aprobado {{
            color: var(--verde-oscuro);
            font-weight: bold;
        }}
        
        .estado-reprobado {{
            color: var(--rojo-oscuro);
            font-weight: bold;
        }}
        
        .clasificaciones {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        
        .clasificacion {{
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease;
        }}
        
        .clasificacion:hover {{
            transform: translateY(-5px);
        }}
        
        .clasificacion-header {{
            padding: 15px;
            color: white;
            text-align: center;
            font-weight: bold;
            font-size: 1.2rem;
        }}
        
        .clasificacion-100 {{
            background-color: var(--verde-oscuro);
        }}
        
        .clasificacion-70 {{
            background-color: var(--amarillo-oscuro);
        }}
        
        .clasificacion-40 {{
            background-color: var(--naranja-oscuro);
        }}
        
        .clasificacion-0 {{
            background-color: var(--rojo-oscuro);
        }}
        
        .asesores-lista {{
            max-height: 400px;
            overflow-y: auto;
            background-color: white;
        }}
        
        .asesor-item {{
            padding: 12px 15px;
            border-bottom: 1px solid #eee;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}
        
        .asesor-item:last-child {{
            border-bottom: none;
        }}
        
        .asesor-nombre {{
            font-weight: 500;
        }}
        
        .asesor-porcentaje {{
            font-weight: bold;
            padding: 4px 10px;
            border-radius: 20px;
            color: white;
            font-size: 0.9rem;
        }}
        
        .gradiente-100 {{
            background: linear-gradient(to right, var(--verde-claro), white);
        }}
        
        .gradiente-70 {{
            background: linear-gradient(to right, var(--amarillo-claro), white);
        }}
        
        .gradiente-40 {{
            background: linear-gradient(to right, var(--naranja-claro), white);
        }}
        
        .gradiente-0 {{
            background: linear-gradient(to right, var(--rojo-claro), white);
        }}
        
        .porcentaje-100 {{
            background-color: var(--verde-oscuro);
        }}
        
        .porcentaje-70 {{
            background-color: var(--amarillo-oscuro);
        }}
        
        .porcentaje-40 {{
            background-color: var(--naranja-oscuro);
        }}
        
        .porcentaje-0 {{
            background-color: var(--rojo-oscuro);
        }}
        
        .estadisticas {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 30px;
        }}
        
        .estadistica-card {{
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }}
        
        .estadistica-valor {{
            font-size: 2rem;
            font-weight: bold;
            margin: 10px 0;
        }}
        
        .estadistica-100 .estadistica-valor {{
            color: var(--verde-oscuro);
        }}
        
        .estadistica-70 .estadistica-valor {{
            color: var(--amarillo-oscuro);
        }}
        
        .estadistica-40 .estadistica-valor {{
            color: var(--naranja-oscuro);
        }}
        
        .estadistica-0 .estadistica-valor {{
            color: var(--rojo-oscuro);
        }}
        
        footer {{
            text-align: center;
            margin-top: 40px;
            padding: 20px;
            color: #666;
            font-size: 0.9rem;
        }}

        /* ESTILOS PARA VISTA DETALLE - FUERA DEL MEDIA QUERY */
        .boton-vista-detalle {{
            background: #2c3e50;
            color: white;
            border: none;
            padding: 12px 25px;
            font-size: 1rem;
            border-radius: 8px;
            cursor: pointer;
            margin: 10px 0;
            transition: background 0.3s ease;
        }}
        
        .boton-vista-detalle:hover {{
            background: #34495e;
        }}
        
        .vista-detalle {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
            margin: 20px 0;
            display: none;
            border: 1px solid #ddd;
        }}
        
        .vista-detalle h2 {{
            color: #2c3e50;
            margin-bottom: 20px;
            text-align: center;
            font-size: 1.5rem;
            font-weight: 600;
        }}
        
        .controles-vista-detalle {{
            background: #f8f9fa;
            padding: 15px;
            border-radius: 6px;
            margin-bottom: 20px;
            border-left: 4px solid #2c3e50;
        }}
        
        .tabla-detalle-container {{
            max-height: 70vh;
            overflow: auto;
            border: 1px solid #e1e5e9;
            border-radius: 6px;
        }}
        
        .tabla-detalle {{
            width: 100%;
            border-collapse: collapse;
            font-size: 0.8rem;
            background: white;
            min-width: 800px;
        }}
        
        .tabla-detalle thead th {{
            background: #2c3e50;
            color: white;
            padding: 12px 8px;
            text-align: center;
            font-weight: 600;
            font-size: 0.75rem;
            position: sticky;
            top: 0;
            z-index: 20;
            border-right: 1px solid #4a6491;
        }}
        
        .tabla-detalle thead th:first-child {{
            border-top-left-radius: 0px;
        }}
        
        .tabla-detalle thead th:last-child {{
            border-top-right-radius: 0px;
            border-right: none;
        }}
        
        .tabla-detalle tbody td {{
            padding: 10px 8px;
            border-bottom: 1px solid #eee;
            text-align: center;
        }}
        
        .tabla-detalle tbody tr:hover td {{
            background: #f8f9fa;
        }}
        
        .team-cell {{
            background: #e3f2fd !important;
            font-weight: 600 !important;
            color: #1565c0 !important;
            position: sticky;
            left: 0;
            z-index: 15;
            border-right: 1px solid #90caf9;
            font-size: 0.75rem;
            min-width: 120px;
        }}
        
        .asesor-cell {{
            background: #f3e5f5 !important;
            font-weight: 600 !important;
            color: #7b1fa2 !important;
            position: sticky;
            left: 120px;
            z-index: 15;
            border-right: 1px solid #ce93d8;
            font-size: 0.75rem;
            min-width: 150px;
        }}
        
        .valor-diario {{
            font-weight: 500;
            padding: 4px 6px;
            border-radius: 4px;
            font-size: 0.75rem;
        }}
        
        .valor-positivo {{
            background: #d4edda;
            color: #155724;
        }}
        
        .valor-cero {{
            background: #f8d7da;
            color: #721c24;
        }}
        
        .valor-alto {{
            background: #28a745;
            color: white;
            font-weight: bold;
        }}
        
        .totales-cell {{
            background: #e3f2fd !important;
            font-weight: 600 !important;
            color: #1565c0 !important;
            border-left: 2px solid #90caf9 !important;
        }}
        
        .alcance-cell {{
            background: #fff3cd !important;
            font-weight: 600 !important;
            color: #856404 !important;
            border-left: 2px solid #f5dca4 !important;
        }}
        
        .resumen-totales {{
            background: #2c3e50;
            color: white;
            padding: 15px;
            border-radius: 6px;
            margin-bottom: 20px;
        }}
        
        .resumen-totales h4 {{
            margin: 0 0 10px 0;
            font-size: 1.1rem;
        }}
        
        .estadisticas-detalle {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-bottom: 20px;
        }}
        
        .estadistica-detalle {{
            background: white;
            padding: 12px;
            border-radius: 6px;
            text-align: center;
            border-left: 4px solid #2c3e50;
        }}
        
        .estadistica-detalle .valor {{
            font-size: 1.3rem;
            font-weight: bold;
            color: #2c3e50;
            margin: 5px 0;
        }}
        
        .estadistica-detalle .label {{
            font-size: 0.8rem;
            color: #6c757d;
        }}

        /* MEDIA QUERY PARA MÓVILES */
        @media (max-width: 768px) {{
            h1 {{
                font-size: 2rem;
            }}
            
            .clasificaciones {{
                grid-template-columns: 1fr;
            }}
            
            .selectores-periodo select {{
                margin: 5px;
                width: 90%;
            }}
            
            .controles-busqueda {{
                flex-direction: column;
            }}
            
            .input-busqueda {{
                min-width: auto;
                width: 100%;
            }}
            
            .tabla-comparacion {{
                font-size: 0.8rem;
            }}
            
            .tabla-comparacion td {{
                padding: 6px 4px;
                min-width: 80px;
            }}
            
            /* Estilos responsive para vista detalle en móviles */
            .vista-detalle {{
                padding: 15px;
            }}
            
            .tabla-detalle-container {{
                max-height: 60vh;
            }}
            
            .tabla-detalle {{
                font-size: 0.7rem;
                min-width: 600px;
            }}
            .sortable {{
                cursor: pointer;
                user-select: none;
                position: relative;
                transition: background-color 0.2s ease;
            }}
            
            .sortable:hover {{
                background-color: #34495e !important;
            }}
            
            .sortable.asc::after {{
                content: ' ▴';
                font-weight: bold;
            }}
            
            .sortable.desc::after {{
                content: ' ▾';
                font-weight: bold;
            }}
            
            /* Ajustar ancho de columnas */
            .team-cell {{
                background: #e3f2fd !important;
                font-weight: 600 !important;
                color: #1565c0 !important;
                position: sticky;
                left: 0;
                z-index: 15;
                border-right: 1px solid #90caf9;
                font-size: 0.75rem;
                min-width: 100px;  /* Reducido de 120px */
                max-width: 120px;
            }}
            
            .asesor-cell {{
                background: #f3e5f5 !important;
                font-weight: 600 !important;
                color: #7b1fa2 !important;
                position: sticky;
                left: 100px;  /* Ajustado según nuevo ancho */
                z-index: 15;
                border-right: 1px solid #ce93d8;
                font-size: 0.75rem;
                min-width: 140px;  /* Reducido de 150px */
                max-width: 160px;
            }}
            
            /* Mejorar visibilidad de valores altos */
            .valor-alto {{
                background: #28a745 !important;
                color: white !important;
                font-weight: bold;
                box-shadow: 0 1px 3px rgba(0,0,0,0.2);
            }}
            
            .valor-cero {{
                background: #f8d7da !important;
                color: #721c24 !important;
                opacity: 0.7;
            }}
        }}
    </style>
</head>
<body>
<div class="container">
    <header>
        <h1>EVOLUTIVO DE ALCANCE POR ASESOR</h1>
        <div class="selectores-periodo">
            <select id="selectorMes" onchange="actualizarPeriodo()">
                {opciones_mes}
            </select>
            <select id="selectorAño" onchange="actualizarPeriodo()">
                {opciones_año}
            </select>
        </div>
        <!-- NUEVO BOTÓN PARA VISTA DETALLE -->
        <button class="boton-vista-detalle" onclick="mostrarVistaDetalle()">
            📊 Ver Detalle por Día
        </button>
        <div class="fecha-actualizacion">Última actualización: {datetime.now().strftime("%d de %B de %Y a las %H:%M")}</div>
    </header>
    
    <div id="contenidoPrincipal">
        <!-- Sección de Clasificación Visual -->
        <div class="clasificaciones">
            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-100">>100%</div>
                <div class="asesores-lista" id="lista-100">
                    {generar_lista_asesores(asesores_data, ">100%")}
                </div>
            </div>
            
            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-70">>70%</div>
                <div class="asesores-lista" id="lista-70">
                    {generar_lista_asesores(asesores_data, ">70%")}
                </div>
            </div>
            
            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-40">>40%</div>
                <div class="asesores-lista" id="lista-40">
                    {generar_lista_asesores(asesores_data, ">40%")}
                </div>
            </div>
            
            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-0">>0%</div>
                <div class="asesores-lista" id="lista-0">
                    {generar_lista_asesores(asesores_data, ">0%")}
                </div>
            </div>
        </div>
        
        <!-- Panel de Estadísticas -->
        <div class="estadisticas">
            <div class="estadistica-card estadistica-100">
                <div>Asesores >100%</div>
                <div class="estadistica-valor" id="cantidad-100">{contadores[">100%"]}</div>
                <div>Meta superada</div>
            </div>
            
            <div class="estadistica-card estadistica-70">
                <div>Asesores >70%</div>
                <div class="estadistica-valor" id="cantidad-70">{contadores[">70%"]}</div>
                <div>Alto rendimiento</div>
            </div>
            
            <div class="estadistica-card estadistica-40">
                <div>Asesores >40%</div>
                <div class="estadistica-valor" id="cantidad-40">{contadores[">40%"]}</div>
                <div>Rendimiento medio</div>
            </div>
            
            <div class="estadistica-card estadistica-0">
                <div>Asesores >0%</div>
                <div class="estadistica-valor" id="cantidad-0">{contadores[">0%"]}</div>
                <div>Por mejorar</div>
            </div>
        </div>
        
        <!-- Sección de Búsqueda y Comparación -->
        <section class="busqueda-asesores">
            <h2>🔍 Búsqueda y Comparación de Asesores</h2>
            <div class="controles-busqueda">
                <input type="text" id="inputBusqueda" class="input-busqueda" placeholder="Escribe el nombre del asesor..." list="listaAsesores">
                <datalist id="listaAsesores">
                    {opciones_asesores}
                </datalist>
                <button class="boton-busqueda" onclick="agregarAsesor()">Agregar Asesor</button>
                <button class="boton-busqueda" onclick="compararAsesores()" style="background: var(--verde-oscuro);">Comparar Seleccionados</button>
                <button class="boton-busqueda" onclick="limpiarBusqueda()" style="background: var(--rojo-oscuro);">Limpiar</button>
            </div>
            
            <div class="asesores-seleccionados" id="asesoresSeleccionados">
                <!-- Aquí se mostrarán los asesores seleccionados -->
            </div>
            
            <div class="resultados-comparacion" id="resultadosComparacion">
                <h3>📊 Comparación de Rendimiento</h3>
                <div class="controles-comparacion">
                    <button class="btn-comparacion" onclick="cambiarVistaComparacion(3)">Últimos 3 meses</button>
                    <button class="btn-comparacion activo" onclick="cambiarVistaComparacion(6)">Últimos 6 meses</button>
                    <button class="btn-comparacion" onclick="cambiarVistaComparacion(10)">Últimos 10 meses</button>
                </div>
                <div id="tablaComparacion"></div>
            </div>
        </section>
        
        <!-- Sección de Periodo de Prueba -->
        <section class="periodo-prueba">
            <h3>📈 Evaluación de Periodo de Prueba</h3>
            <div class="controles-periodo">
                <select id="mes1" class="input-periodo">
                    <option value="">Mes 1</option>
                    {opciones_mes}
                </select>
                <select id="año1" class="input-periodo">
                    <option value="">Año 1</option>
                    {opciones_año}
                </select>
                <select id="mes2" class="input-periodo">
                    <option value="">Mes 2</option>
                    {opciones_mes}
                </select>
                <select id="año2" class="input-periodo">
                    <option value="">Año 2</option>
                    {opciones_año}
                </select>
                <input type="number" id="porcentajeMinimo" class="input-periodo" placeholder="% Mínimo" value="70" min="0" max="100" step="0.1">
            </div>
            
            <div class="controles-busqueda" style="margin-top: 15px;">
                <input type="text" id="inputBusquedaPeriodo" class="input-busqueda" placeholder="Escribe el nombre del asesor para el periodo de prueba..." list="listaAsesores">
                <button class="boton-busqueda" onclick="agregarAsesorPeriodo()">Agregar Asesor</button>
                <button class="boton-busqueda" onclick="seleccionarTodosAsesoresPeriodo()" style="background: var(--verde-oscuro);">Seleccionar Todos</button>
                <button class="boton-busqueda" onclick="calcularPeriodoPrueba()" style="background: var(--azul-oscuro);">Calcular Periodo de Prueba</button>
                <button class="boton-busqueda" onclick="limpiarPeriodoPrueba()" style="background: var(--rojo-oscuro);">Limpiar Todo</button>
            </div>
            
            <div class="asesores-seleccionados" id="asesoresSeleccionadosPeriodo">
                <!-- Aquí se mostrarán los asesores seleccionados para periodo de prueba -->
            </div>
            
            <div class="resultados-periodo" id="resultadosPeriodo">
                <div id="tablaPeriodo"></div>
            </div>
        </section>
    </div>
    <!-- NUEVA SECCIÓN MEJORADA PARA VISTA DETALLE -->
    <section class="vista-detalle" id="vistaDetalle">
        <h2>📊 Detalle de Recupero Diario</h2>
        
        <div class="controles-vista-detalle">
            <div class="selectores-periodo">
                <select id="selectorMesDetalle" onchange="actualizarVistaDetalle()">
                    {opciones_mes}
                </select>
                <select id="selectorAñoDetalle" onchange="actualizarVistaDetalle()">
                    {opciones_año}
                </select>
            </div>
        </div>
        
        <div class="resumen-totales">
            <h4>Resumen del Mes: <span id="mesActualDetalle">{mes_actual} {año_actual}</span></h4>
            <div class="estadisticas-detalle" id="estadisticasDetalle">
                <!-- Las estadísticas se cargarán dinámicamente -->
            </div>
        </div>
        
        <div class="tabla-detalle-container">
            <div id="tablaDetalle"></div>
        </div>
    </section>
    
    <footer>
        <p>Sistema de seguimiento de metas - Actualización diaria</p>
    </footer>
</div>

<script>
    // Datos de todos los meses (precargados)
    const datosMeses = {json.dumps(todos_los_datos, default=str, ensure_ascii=False)};
    const asesoresSeleccionados = new Set();
    let mesesAMostrar = 6; // Por defecto últimos 6 meses
    
    function actualizarPeriodo() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        // Verificar si existe datos para este periodo
        if (!datosMeses[periodoCompleto]) {{
            alert('No hay datos disponibles para ' + periodoCompleto.replace('_', ' '));
            return;
        }}
        
        // Actualizar los datos mostrados
        const asesores = datosMeses[periodoCompleto] || [];
        
        // Actualizar cada lista
        actualizarLista(asesores, '>100%', 'lista-100');
        actualizarLista(asesores, '>70%', 'lista-70');
        actualizarLista(asesores, '>40%', 'lista-40');
        actualizarLista(asesores, '>0%', 'lista-0');
        
        // Actualizar estadísticas
        actualizarEstadisticas(asesores);
    }}
    
    function actualizarLista(asesores, clasificacion, idLista) {{
        const lista = document.getElementById(idLista);
        const asesoresFiltrados = asesores.filter(a => a.clasificacion === clasificacion);
        
        if (asesoresFiltrados.length === 0) {{
            lista.innerHTML = '<div class="asesor-item">No hay asesores en esta categoría</div>';
            return;
        }}
        
        // Ordenar por porcentaje (mayor a menor)
        asesoresFiltrados.sort((a, b) => b.porcentaje - a.porcentaje);
        
        let html = '';
        asesoresFiltrados.forEach(asesor => {{
            const claseGradiente = `gradiente-${{clasificacion.replace('%', '').replace('>', '')}}`;
            const clasePorcentaje = `porcentaje-${{clasificacion.replace('%', '').replace('>', '')}}`;
            html += `<div class="asesor-item ${{claseGradiente}}">
                <div class="asesor-nombre">${{asesor.nombre}}</div>
                <div class="asesor-porcentaje ${{clasePorcentaje}}">${{asesor.porcentaje}}%</div>
            </div>`;
        }});
        
        lista.innerHTML = html;
    }}
    
    function actualizarEstadisticas(asesores) {{
        const contadores = {{
            ">100%": asesores.filter(a => a.clasificacion === ">100%").length,
            ">70%": asesores.filter(a => a.clasificacion === ">70%").length,
            ">40%": asesores.filter(a => a.clasificacion === ">40%").length,
            ">0%": asesores.filter(a => a.clasificacion === ">0%").length
        }};
        
        document.getElementById('cantidad-100').textContent = contadores[">100%"];
        document.getElementById('cantidad-70').textContent = contadores[">70%"];
        document.getElementById('cantidad-40').textContent = contadores[">40%"];
        document.getElementById('cantidad-0').textContent = contadores[">0%"];
    }}
    
    // Funciones para la búsqueda y comparación
    function agregarAsesor() {{
        const input = document.getElementById('inputBusqueda');
        const nombreAsesor = input.value.trim();
        
        if (nombreAsesor && !asesoresSeleccionados.has(nombreAsesor)) {{
            asesoresSeleccionados.add(nombreAsesor);
            actualizarAsesoresSeleccionados();
            input.value = '';
        }}
    }}
    
    function eliminarAsesor(nombre) {{
        asesoresSeleccionados.delete(nombre);
        actualizarAsesoresSeleccionados();
    }}
    
    function actualizarAsesoresSeleccionados() {{
        const container = document.getElementById('asesoresSeleccionados');
        container.innerHTML = '';
        
        asesoresSeleccionados.forEach(asesor => {{
            const tag = document.createElement('div');
            tag.className = 'tag-asesor';
            tag.innerHTML = `
                ${{asesor}}
                <button class="eliminar-asesor" onclick="eliminarAsesor('${{asesor}}')">×</button>
            `;
            container.appendChild(tag);
        }});
    }}
    
    function cambiarVistaComparacion(cantidadMeses) {{
        mesesAMostrar = cantidadMeses;
        
        // Actualizar botones activos
        document.querySelectorAll('.btn-comparacion').forEach(btn => {{
            btn.classList.remove('activo');
        }});
        event.target.classList.add('activo');
        
        // Regenerar tabla si ya hay asesores seleccionados
        if (asesoresSeleccionados.size > 0) {{
            compararAsesores();
        }}
    }}
    
    function compararAsesores() {{
        if (asesoresSeleccionados.size === 0) {{
            alert('Por favor selecciona al menos un asesor para comparar.');
            return;
        }}

        const resultadosDiv = document.getElementById('resultadosComparacion');
        const tablaDiv = document.getElementById('tablaComparacion');

        // Obtener meses ordenados (más reciente primero) y EXCLUIR el mes actual
        let mesesOrdenados = Object.keys(datosMeses).sort((a, b) => {{
            const [mesA, añoA] = a.split('_');
            const [mesB, añoB] = b.split('_');
            const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
            const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
            return fechaB - fechaA; // Orden descendente (más reciente primero)
        }});

        // Excluir el mes actual (el más reciente)
        if (mesesOrdenados.length > 0) {{
            mesesOrdenados = mesesOrdenados.slice(1);
        }}

        // Filtrar por cantidad de meses (sin contar el actual)
        if (mesesAMostrar > 0 && mesesOrdenados.length > mesesAMostrar) {{
            mesesOrdenados = mesesOrdenados.slice(0, mesesAMostrar);
        }}

        // Crear tabla de comparación INVERTIDA (asesores en filas, meses en columnas)
        let html = '<table class="tabla-comparacion">';

        // Encabezado con meses
        html += '<tr><th class="mes-header">Asesor</th>';
        mesesOrdenados.forEach(mes => {{
            html += `<th class="mes-header">${{mes.replace('_', ' ')}}</th>`;
        }});
        html += '</tr>';

        // Filas por cada asesor (SOLO UNA FILA POR ASESOR, SIN SUPERVISOR)
        asesoresSeleccionados.forEach(asesor => {{
            html += `<tr><td class="asesor-name">${{asesor}}</td>`;
    
            mesesOrdenados.forEach(mes => {{
                const datosMes = datosMeses[mes];
                const asesorData = datosMes.find(a => a.nombre === asesor);
        
                if (asesorData) {{
                    const clasePorcentaje = `porcentaje-tabla porcentaje-${{asesorData.clasificacion.replace('%', '').replace('>', '')}}`;
                    html += `<td><span class="${{clasePorcentaje}}">${{asesorData.porcentaje}}%</span></td>`;
                }} else {{
                    html += '<td class="sin-participacion">No participó</td>';
                }}
            }});
    
            html += '</tr>';
        }});

        html += '</table>';
        tablaDiv.innerHTML = html;
        resultadosDiv.style.display = 'block';
    }}
    
    function limpiarBusqueda() {{
        asesoresSeleccionados.clear();
        actualizarAsesoresSeleccionados();
        document.getElementById('resultadosComparacion').style.display = 'none';
        document.getElementById('inputBusqueda').value = '';
    }}
    
    // Variables para Periodo de Prueba
    const asesoresSeleccionadosPeriodo = new Set();
    
    // Funciones para Periodo de Prueba
    function agregarAsesorPeriodo() {{
        const input = document.getElementById('inputBusquedaPeriodo');
        const nombreAsesor = input.value.trim();
        
        if (nombreAsesor && !asesoresSeleccionadosPeriodo.has(nombreAsesor)) {{
            asesoresSeleccionadosPeriodo.add(nombreAsesor);
            actualizarAsesoresSeleccionadosPeriodo();
            input.value = '';
        }}
    }}
    
    function eliminarAsesorPeriodo(nombre) {{
        asesoresSeleccionadosPeriodo.delete(nombre);
        actualizarAsesoresSeleccionadosPeriodo();
    }}
    
    function actualizarAsesoresSeleccionadosPeriodo() {{
        const container = document.getElementById('asesoresSeleccionadosPeriodo');
        container.innerHTML = '';
        
        asesoresSeleccionadosPeriodo.forEach(asesor => {{
            const tag = document.createElement('div');
            tag.className = 'tag-asesor';
            tag.innerHTML = `
                ${{asesor}}
                <button class="eliminar-asesor" onclick="eliminarAsesorPeriodo('${{asesor}}')">×</button>
            `;
            container.appendChild(tag);
        }});
    }}
    
    function seleccionarTodosAsesoresPeriodo() {{
        // Obtener todos los asesores únicos de todos los meses
        const todosAsesores = new Set();
        Object.values(datosMeses).forEach(mesData => {{
            mesData.forEach(asesor => {{
                todosAsesores.add(asesor.nombre);
            }});
        }});
        
        asesoresSeleccionadosPeriodo.clear();
        todosAsesores.forEach(asesor => {{
            asesoresSeleccionadosPeriodo.add(asesor);
        }});
        actualizarAsesoresSeleccionadosPeriodo();
    }}
    
    function limpiarPeriodoPrueba() {{
        document.getElementById('mes1').value = '';
        document.getElementById('año1').value = '';
        document.getElementById('mes2').value = '';
        document.getElementById('año2').value = '';
        document.getElementById('porcentajeMinimo').value = '70';
        document.getElementById('inputBusquedaPeriodo').value = '';
        asesoresSeleccionadosPeriodo.clear();
        actualizarAsesoresSeleccionadosPeriodo();
        document.getElementById('resultadosPeriodo').style.display = 'none';
    }}
    
    function calcularPeriodoPrueba() {{
        const mes1 = document.getElementById('mes1').value;
        const año1 = document.getElementById('año1').value;
        const mes2 = document.getElementById('mes2').value;
        const año2 = document.getElementById('año2').value;
        const porcentajeMinimo = parseFloat(document.getElementById('porcentajeMinimo').value) || 70;
        
        if (!mes1 || !año1 || !mes2 || !año2) {{
            alert('Por favor selecciona ambos meses y años para calcular el periodo de prueba.');
            return;
        }}
        
        const periodo1 = `${{mes1}}_${{año1}}`;
        const periodo2 = `${{mes2}}_${{año2}}`;
        
        if (!datosMeses[periodo1] || !datosMeses[periodo2]) {{
            alert('No hay datos disponibles para uno de los periodos seleccionados.');
            return;
        }}
        
        const resultadosDiv = document.getElementById('resultadosPeriodo');
        const tablaDiv = document.getElementById('tablaPeriodo');
        
        // Obtener asesores a evaluar (seleccionados o todos)
        const asesoresAEvaluar = asesoresSeleccionadosPeriodo.size > 0 ? 
            Array.from(asesoresSeleccionadosPeriodo) : 
            [...new Set([...datosMeses[periodo1].map(a => a.nombre), ...datosMeses[periodo2].map(a => a.nombre)])];
        
        let html = '<table class="tabla-periodo">';
        html += '<tr><th>Asesor</th><th>Periodo 1</th><th>Periodo 2</th><th>Total Recupero</th><th>Total Meta</th><th>% Periodo Prueba</th><th>Estado</th><th>Falta Recuperar</th></tr>';
        
        asesoresAEvaluar.forEach(asesor => {{
            const data1 = datosMeses[periodo1].find(a => a.nombre === asesor);
            const data2 = datosMeses[periodo2].find(a => a.nombre === asesor);
            
            if (data1 && data2 && data1.recupero !== undefined && data1.meta !== undefined && 
                data2.recupero !== undefined && data2.meta !== undefined) {{
                
                const totalRecupero = data1.recupero + data2.recupero;
                const totalMeta = data1.meta + data2.meta;
                const porcentajePrueba = totalMeta > 0 ? (totalRecupero / totalMeta) * 100 : 0;
                const porcentajeFormateado = porcentajePrueba.toFixed(2);
                
                const estado = porcentajePrueba >= porcentajeMinimo ? 
                    '<span class="estado-aprobado">✓ Supera</span>' : 
                    '<span class="estado-reprobado">✗ No supera</span>';
                
                const faltaRecuperar = porcentajePrueba < porcentajeMinimo ? 
                    (totalMeta * (porcentajeMinimo / 100) - totalRecupero).toFixed(2) : '0';
                
                html += `<tr>
                    <td>${{asesor}}</td>
                    <td>${{data1.recupero}} / ${{data1.meta}}</td>
                    <td>${{data2.recupero}} / ${{data2.meta}}</td>
                    <td><strong>${{totalRecupero.toFixed(2)}}</strong></td>
                    <td><strong>${{totalMeta.toFixed(2)}}</strong></td>
                    <td><strong>${{porcentajeFormateado}}%</strong></td>
                    <td>${{estado}}</td>
                    <td>${{faltaRecuperar}}</td>
                </tr>`;
            }}
        }});
        
        html += '</table>';
        tablaDiv.innerHTML = html;
        resultadosDiv.style.display = 'block';
    }}
    
    // Permitir agregar asesor con Enter en el periodo de prueba
    document.getElementById('inputBusquedaPeriodo').addEventListener('keypress', function(e) {{
        if (e.key === 'Enter') {{
            agregarAsesorPeriodo();
        }}
    }});
    
    function obtenerNumeroMes(mes) {{
        const meses = {{
            'ENERO': 0, 'FEBRERO': 1, 'MARZO': 2, 'ABRIL': 3, 'MAYO': 4, 'JUNIO': 5,
            'JULIO': 6, 'AGOSTO': 7, 'SETIEMBRE': 8, 'SEPTIEMBRE': 8, 
            'OCTUBRE': 9, 'NOVIEMBRE': 10, 'DICIEMBRE': 11
        }};
        return meses[mes.toUpperCase()] || 0;
    }}
    
    // Permitir agregar asesor con Enter
    document.getElementById('inputBusqueda').addEventListener('keypress', function(e) {{
        if (e.key === 'Enter') {{
            agregarAsesor();
        }}
    }});

    // Función para mostrar/ocultar la vista detalle
    function mostrarVistaDetalle() {{
        const vistaDetalle = document.getElementById('vistaDetalle');
        const contenidoPrincipal = document.getElementById('contenidoPrincipal');
        
        if (vistaDetalle.style.display === 'none') {{
            // Mostrar vista detalle y ocultar contenido principal
            vistaDetalle.style.display = 'block';
            contenidoPrincipal.style.display = 'none';
            actualizarVistaDetalle();
        }} else {{
            // Ocultar vista detalle y mostrar contenido principal
            vistaDetalle.style.display = 'none';
            contenidoPrincipal.style.display = 'block';
        }}
    }}
    
    // Función para actualizar la vista detalle
    function actualizarVistaDetalle() {{
        const mesSeleccionado = document.getElementById('selectorMesDetalle').value;
        const añoSeleccionado = document.getElementById('selectorAñoDetalle').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        // Actualizar el título
        document.getElementById('mesActualDetalle').textContent = `${{mesSeleccionado}} ${{añoSeleccionado}}`;
        
        // Verificar si existe datos para este periodo
        if (!datosMeses[periodoCompleto]) {{
            document.getElementById('tablaDetalle').innerHTML = 
                '<div style="text-align: center; padding: 40px; color: #6c757d;">' +
                '<h3>No hay datos disponibles para ' + periodoCompleto.replace('_', ' ') + '</h3>' +
                '<p>Selecciona otro mes o año</p></div>';
            document.getElementById('estadisticasDetalle').innerHTML = '';
            return;
        }}
        
        const asesores = datosMeses[periodoCompleto] || [];
        
        // Calcular estadísticas
        calcularEstadisticasDetalle(asesores);
        
        // Obtener todas las fechas únicas de los datos diarios
        const todasFechas = new Set();
        asesores.forEach(asesor => {{
            Object.keys(asesor.datos_diarios || {{}}).forEach(fecha => {{
                todasFechas.add(fecha);
            }});
        }});
        
        // Ordenar fechas
        let fechasOrdenadas = Array.from(todasFechas).sort((a, b) => {{
            return new Date(a.split('-').reverse().join('-')) - new Date(b.split('-').reverse().join('-'));
        }});
        
        // FILTRAR FECHAS: Mostrar solo hasta ayer (o la fecha más reciente disponible)
        const hoy = new Date();
        const ayer = new Date(hoy);
        ayer.setDate(hoy.getDate() - 1);
        
        // Formatear ayer como "d-Mes" (ej: "28-Nov")
        const formatoAyer = ayer.getDate() + '-' + ayer.toLocaleString('es', {{ month: 'short' }}).toUpperCase();
        
        // Encontrar la fecha más cercana a ayer que exista en los datos
        let fechaLimite = null;
        for (let i = fechasOrdenadas.length - 1; i >= 0; i--) {{
            const fechaData = fechasOrdenadas[i];
            // Intentar encontrar una fecha igual o anterior a ayer
            if (fechaLimite === null) {{
                fechaLimite = fechaData;
            }}
            // Si encontramos una fecha que coincide con ayer, usarla
            if (fechaData === formatoAyer) {{
                fechaLimite = fechaData;
                break;
            }}
        }}
        
        // Filtrar fechas para mostrar solo hasta la fecha límite
        if (fechaLimite) {{
            const indiceLimite = fechasOrdenadas.indexOf(fechaLimite);
            if (indiceLimite !== -1) {{
                fechasOrdenadas = fechasOrdenadas.slice(0, indiceLimite + 1);
            }}
        }}
        
        // Crear tabla detalle con capacidad de ordenamiento
        let html = '<table class="tabla-detalle" id="tablaDetalleData">';
        
        // Encabezado con botones de ordenamiento
        html += '<thead><tr>';
        html += '<th class="sortable" data-sort="equipo">Equipo ▾</th>';
        html += '<th class="sortable" data-sort="asesor">Asesor ▾</th>';
        
        // Agregar columnas de fechas
        fechasOrdenadas.forEach(fecha => {{
            html += '<th>' + fecha + '</th>';
        }});
        
        html += '<th class="sortable" data-sort="recupero">Recupero Total ▾</th>';
        html += '<th class="sortable" data-sort="alcance">% Alcance ▾</th>';
        html += '</tr></thead>';
        
        html += '<tbody>';
        
        // Ahora NO agrupamos por supervisor - cada asesor en su propia fila
        asesores.forEach((asesor, index) => {{
            html += '<tr>';
            html += '<td class="team-cell">' + (asesor.supervisor || 'Sin Supervisor') + '</td>';
            html += '<td class="asesor-cell">' + asesor.nombre + '</td>';
            
            // Datos diarios
            fechasOrdenadas.forEach(fecha => {{
                const valor = asesor.datos_diarios ? (asesor.datos_diarios[fecha] || 0) : 0;
                let claseValor = 'valor-cero';
                if (valor > 0) {{
                    claseValor = valor > 500 ? 'valor-alto' : 'valor-positivo'; // Ajusta 500 según tu criterio
                }}
                html += '<td><span class="valor-diario ' + claseValor + '">' + valor.toFixed(2) + '</span></td>';
            }});
            
            // Recupero total y alcance
            html += '<td class="totales-cell" data-value="' + (asesor.recupero || 0) + '">' + (asesor.recupero !== undefined ? asesor.recupero.toFixed(2) : '0.00') + '</td>';
            html += '<td class="alcance-cell" data-value="' + asesor.porcentaje + '">' + asesor.porcentaje + '%</td>';
            
            html += '</tr>';
        }});
        
        html += '</tbody></table>';
        document.getElementById('tablaDetalle').innerHTML = html;
        
        // Agregar event listeners para ordenamiento
        agregarFuncionalidadOrdenamiento();
    }}
    
    function calcularEstadisticasDetalle(asesores) {{
        const totalAsesores = asesores.length;
        const totalRecupero = asesores.reduce((sum, asesor) => sum + (asesor.recupero || 0), 0);
        const totalMeta = asesores.reduce((sum, asesor) => sum + (asesor.meta || 0), 0);
        const alcancePromedio = totalMeta > 0 ? (totalRecupero / totalMeta) * 100 : 0;
        
        const html = 
            '<div class="estadistica-detalle">' +
            '<div class="valor">' + totalAsesores + '</div>' +
            '<div class="label">Total Asesores</div>' +
            '</div>' +
            '<div class="estadistica-detalle">' +
            '<div class="valor">S/ ' + totalRecupero.toFixed(2) + '</div>' +
            '<div class="label">Recupero Total</div>' +
            '</div>' +
            '<div class="estadistica-detalle">' +
            '<div class="valor">S/ ' + totalMeta.toFixed(2) + '</div>' +
            '<div class="label">Meta Total</div>' +
            '</div>' +
            '<div class="estadistica-detalle">' +
            '<div class="valor">' + alcancePromedio.toFixed(1) + '%</div>' +
            '<div class="label">Alcance Promedio</div>' +
            '</div>';
        
        document.getElementById('estadisticasDetalle').innerHTML = html;
    }}
    
    // NUEVA FUNCIÓN DE ORDENAMIENTO - FUERA de actualizarVistaDetalle
    function agregarFuncionalidadOrdenamiento() {{
        const headers = document.querySelectorAll('#tablaDetalleData .sortable');
        
        headers.forEach(header => {{
            header.addEventListener('click', () => {{
                const tipo = header.getAttribute('data-sort');
                const tbody = document.querySelector('#tablaDetalleData tbody');
                const filas = Array.from(tbody.querySelectorAll('tr'));
                const isAsc = !header.classList.contains('asc');
                
                // Remover clases de todos los headers
                headers.forEach(h => {{
                    h.classList.remove('asc', 'desc');
                    h.innerHTML = h.innerHTML.replace('▴', '▾').replace('▾', '▾');
                }});
                
                // Ordenar filas
                filas.sort((a, b) => {{
                    let valorA, valorB;
                    
                    switch(tipo) {{
                        case 'equipo':
                            valorA = a.cells[0].textContent.toLowerCase();
                            valorB = b.cells[0].textContent.toLowerCase();
                            break;
                        case 'asesor':
                            valorA = a.cells[1].textContent.toLowerCase();
                            valorB = b.cells[1].textContent.toLowerCase();
                            break;
                        case 'recupero':
                            valorA = parseFloat(a.cells[a.cells.length - 2].getAttribute('data-value'));
                            valorB = parseFloat(b.cells[b.cells.length - 2].getAttribute('data-value'));
                            break;
                        case 'alcance':
                            valorA = parseFloat(a.cells[a.cells.length - 1].getAttribute('data-value'));
                            valorB = parseFloat(b.cells[b.cells.length - 1].getAttribute('data-value'));
                            break;
                        default:
                            return 0;
                    }}
                    
                    if (typeof valorA === 'string') {{
                        return isAsc ? valorA.localeCompare(valorB) : valorB.localeCompare(valorA);
                    }} else {{
                        return isAsc ? valorB - valorA : valorA - valorB; // Desc por defecto para números
                    }}
                }});
                
                // Reinsertar filas ordenadas
                filas.forEach(fila => tbody.appendChild(fila));
                
                // Actualizar indicador visual
                header.classList.add(isAsc ? 'asc' : 'desc');
                header.innerHTML = header.innerHTML.replace('▾', isAsc ? '▴' : '▾');
            }});
        }});
    }}
    
    // Inicializar selectores de vista detalle con los valores actuales
    document.getElementById('selectorMesDetalle').value = document.getElementById('selectorMes').value;
    document.getElementById('selectorAñoDetalle').value = document.getElementById('selectorAño').value;
</script>
</body>
</html>'''
    
    with open(ruta_html, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML actualizado y guardado en: {ruta_html}")

def obtener_mes_año_actual(meses_validos):
    """Obtiene el mes y año más reciente disponible"""
    if not meses_validos:
        return "NOVIEMBRE", "2025"
    
    # Ordenar por fecha (más reciente primero)
    meses_ordenados = sorted(meses_validos, key=lambda x: convertir_mes_a_fecha(x), reverse=True)
    mes, año = meses_ordenados[0].split('_')
    return mes, año

def convertir_mes_a_fecha(nombre_mes):
    """Convierte nombre de mes_año a fecha para ordenar"""
    try:
        mes, año = nombre_mes.split('_')
        # Mapear nombres de meses en español
        meses = {
            'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
            'JULIO': 7, 'AGOSTO': 8, 'SETIEMBRE': 9, 'SEPTIEMBRE': 9, 
            'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
        }
        return datetime(int(año), meses.get(mes.upper(), 1), 1)
    except:
        return datetime(1900, 1, 1)

def generar_lista_asesores(asesores_data, clasificacion):
    asesores_filtrados = [a for a in asesores_data if a['clasificacion'] == clasificacion]
    
    if not asesores_filtrados:
        return '<div class="asesor-item">No hay asesores en esta categoría</div>'
    
    html_items = []
    for asesor in asesores_filtrados:
        clase_gradiente = f"gradiente-{clasificacion.replace('%', '').replace('>', '')}"
        clase_porcentaje = f"porcentaje-{clasificacion.replace('%', '').replace('>', '')}"
        
        item_html = f'''
        <div class="asesor-item {clase_gradiente}">
            <div class="asesor-nombre">{asesor['nombre']}</div>
            <div class="asesor-porcentaje {clase_porcentaje}">
                {asesor['porcentaje']}%
            </div>
        </div>'''
        html_items.append(item_html)
    
    return ''.join(html_items)

def realizar_git_operations():
    try:
        subprocess.run(["git", "add", "index.html"], check=True)
        print("Archivo index.html agregado al staging area de Git")
        
        commit_message = f"Actualización automática - {datetime.now().strftime('%Y-%m-%d %H:%M')}"
        subprocess.run(["git", "commit", "-m", commit_message], check=True)
        print("Commit realizado")
        
        subprocess.run(["git", "push"], check=True)
        print("Push al repositorio remoto completado")
        
    except subprocess.CalledProcessError as e:
        print(f"Error en operaciones Git: {e}")
    except Exception as e:
        print(f"Error inesperado en Git: {e}")

if __name__ == "__main__":
    procesar_excel_y_actualizar_html()

Trabajando desde: C:\Users\Jorge Vasquez\Ranking
Excel: C:\Users\Jorge Vasquez\Ranking\CS_AVANCE DE ASESORES.xlsx
HTML: C:\Users\Jorge Vasquez\Ranking\index.html
Hojas disponibles: ['ENERO_2025', 'FEBRERO_2025', 'MARZO_2025', 'ABRIL_2025', 'MAYO_2025', 'JUNIO_2025', 'JULIO_2025', 'AGOSTO_2025', 'SETIEMBRE_2025', 'OCTUBRE_2025', 'NOVIEMBRE_2025']
Meses válidos encontrados: ['ENERO_2025', 'FEBRERO_2025', 'MARZO_2025', 'ABRIL_2025', 'MAYO_2025', 'JUNIO_2025', 'JULIO_2025', 'AGOSTO_2025', 'SETIEMBRE_2025', 'OCTUBRE_2025', 'NOVIEMBRE_2025']
Años disponibles: ['2025']
Meses disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SETIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']

Procesando hoja: ENERO_2025
  - 70 asesores procesados

Procesando hoja: FEBRERO_2025
  - 70 asesores procesados

Procesando hoja: MARZO_2025
  - 72 asesores procesados

Procesando hoja: ABRIL_2025
  - 67 asesores procesados

Procesando hoja: MAYO_2025
  - 76 asesores procesados

Proce